In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('data.csv')

In [3]:
target = np.zeros(len(data))
for index in range(len(data)-1):
    cur = data.iloc[index, 2]
    nxt = data.iloc[index+1, 2]
    
    if nxt >= cur:
        target[index] = 1
    else:
        target[index] = 0

In [4]:
data['target'] = target

In [5]:
data.describe()

,Open Price,Close Price,High Price,Low Price,Volume,target
count,2516.000000,2516.000000,2516.000000,2516.000000,2.516000e+03,2516.000000
mean,1765.493029,1765.867067,1774.433740,1755.869141,2.869158e+09,0.545310
std,566.003293,565.538751,566.671163,564.789277,1.312045e+09,0.498042
min,679.280000,676.530000,695.270000,666.790000,5.181584e+08,0.000000
25%,1278.175000,1278.315000,1286.160000,1268.235000,2.016047e+09,0.000000
50%,1806.270000,1805.450000,1810.450000,1800.675000,2.425113e+09,1.000000
75%,2122.492500,2124.222500,2129.150000,2113.325000,3.248909e+09,1.000000
max,2936.760000,2930.750000,2940.910000,2927.110000,9.120100e+09,1.000000


In [6]:
data.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume,target
0,02-Jan-2009,902.99,931.80,934.73,899.35,4048270080,0.0
1,05-Jan-2009,929.17,927.45,936.63,919.53,5413910016,1.0
2,06-Jan-2009,931.17,934.70,943.85,927.28,5392620032,0.0
3,07-Jan-2009,927.45,906.65,927.45,902.37,4704940032,1.0
4,08-Jan-2009,905.73,909.73,910.00,896.81,4991549952,0.0


In [7]:
data['Date'] = pd.to_datetime(data['Date'])

In [8]:
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['day'] = data['Date'].dt.day
data['day_of_week'] = data['Date'].dt.dayofweek

In [9]:
# plt.figure(figsize=(14,5))
# plt.plot(data['Close Price'])

In [10]:
data.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume,target,year,month,day,day_of_week
0,2009-01-02,902.99,931.80,934.73,899.35,4048270080,0.0,2009,1,2,4
1,2009-01-05,929.17,927.45,936.63,919.53,5413910016,1.0,2009,1,5,0
2,2009-01-06,931.17,934.70,943.85,927.28,5392620032,0.0,2009,1,6,1
3,2009-01-07,927.45,906.65,927.45,902.37,4704940032,1.0,2009,1,7,2
4,2009-01-08,905.73,909.73,910.00,896.81,4991549952,0.0,2009,1,8,3


In [11]:
train = data[data['year'] < 2018]
test = data[data['year'] == 2018]

In [12]:
train.shape

(2264, 11)

In [13]:
test.shape

(252, 11)

In [14]:
train_x = train[['Open Price', 'Close Price', 'High Price', 'Volume', 'year', 'month', 'day', 'day_of_week']]
train_y = train['target']
test_x = test[['Open Price', 'Close Price', 'High Price', 'Volume', 'year', 'month', 'day', 'day_of_week']]
test_y = test['target']

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_x)

std_train_x = scaler.transform(train_x)
std_test_x = scaler.transform(test_x)

### Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

logi_model = LogisticRegression()
logi_model.fit(std_train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
logi_model.score(std_test_x, test_y)

0.5396825396825397

In [18]:
from sklearn import svm

svm_model = svm.SVC(tol=0.1)
svm_model.fit(std_train_x, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.1, verbose=False)

In [19]:
from sklearn.metrics import accuracy_score

prediction = svm_model.predict(std_test_x)
print(accuracy_score(test_y, prediction))

0.5238095238095238


In [20]:
from keras import models, layers, optimizers, regularizers

Using TensorFlow backend.


In [21]:
model = models.Sequential()

model.add(layers.Dense(input_dim=8,
                       units=20, 
                       activation='relu'))
# model.add(layers.Dense(units=20, 
#                        activation='relu'))
model.add(layers.Dense(units=1,
                       activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()
model.fit(std_train_x, train_y,epochs=10)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                180       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2264/2264 [==============================] - 2s 666us/step - loss: 0.7004 - acc: 0.5225
Epoch 2/10
2264/2264 [==============================] - 0s 111us/step - loss: 0.6921 - acc: 0.5398
Epoch 3/10
2264/2264 [==============================] - 0s 103us/step - loss: 0.6902 - acc: 0.5424
Epoch 4/10
2264/2264 [==============================] - 0s 99us/step - loss: 0.6891 - acc: 0.5455
Epoch 5/10
2264/2264 [===

In [22]:
model.evaluate(std_test_x,test_y)

252/252 [==============================] - 0s 1ms/step


[0.6975364070090037, 0.5000000009461055]